In [ ]:
import pandas as pd
import json
import pickle
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import os,sys,inspect, pickle
import numpy as np

from utils import *

pd.set_option('display.max_colwidth', 500)

In [ ]:
def create_empty_base_df():
    data = {
        'Algorithm': [],
        'Runtime total': [],
        'Runtime index simulation': [],
        'Runtime cost requests': [],
        'Cost Requests Total': [],
        'Not cached': [],
        'Cost Requests Cached': [],
        'Evaluated Configurations': [],
        'Indexes Simulated': []
    }
    
    return pd.DataFrame(data, columns = data.keys())


def best_row_fitting_budget(df, budget):
    df['original idx'] = df.index
    df = df[df['memory consumption'] < budget]
    if len(get_costs(df)) < 1:
        return None
    row_with_best_configuration = get_costs(df).index(min(get_costs(df)))
    return df.iloc[row_with_best_configuration]

In [ ]:
CSV_PATH = '../tpcds_wo_4_6_9_10_11_32_35_41_95/'
BENCHMARK = 'tpcds'
SCALE_FACTOR = 10
# all except queries that dominate costs by orders of magnitude
QUERIES = [1, 2, 3, 5, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99]

BUDGETS_B = [gb_to_b(5)]

In [ ]:
result = create_empty_base_df()
budget = BUDGETS_B[0]
for algorithm in ALGORITHMS:
    if algorithm == 'cophy' or algorithm == 'dexter':
        continue
        
    style = styles[algorithm]
    csv_path= f"{CSV_PATH}/results_{algorithm}_{BENCHMARK}_{len(QUERIES)}_queries.csv"
    try:
        df = pd.read_csv(csv_path, sep=';')
    except:
        continue
    if algorithm == 'dexter':
        df = df.iloc[::-1]
    best_fitting_row = best_row_fitting_budget(df, budget)
    if best_fitting_row is not None:
        print(best_fitting_row['parameters'])
        unique_workload_evaluations = int((best_fitting_row['cost requests'] - best_fitting_row['cache hits']) / len(QUERIES))
        result = result.append({
            'Algorithm': style.label,
            'Runtime total': f"{s_to_m(best_fitting_row['algorithm runtime']):.2f}",
            'Runtime index simulation': f"{best_fitting_row['algorithm index creation time'] / best_fitting_row['algorithm runtime'] * 100:.2f}",
            'Runtime cost requests': f"{best_fitting_row['algorithm cost time'] / best_fitting_row['algorithm runtime'] * 100:.2f}",
            'Cost Requests Total': f"{best_fitting_row['cost requests']:d}",
            'Not cached': f"{best_fitting_row['cost requests']-best_fitting_row['cache hits']:d}",
            'Cost Requests Cached': f"{best_fitting_row['cache hits'] / best_fitting_row['cost requests'] * 100 if best_fitting_row['cache hits'] > 0 else 0 : .2f}",
            'Evaluated Configurations': f"{unique_workload_evaluations:d}",
            'Indexes Simulated': f"{best_fitting_row['algorithm created #indexes']:d}",
        }, ignore_index=True)

# Dexter must be hardcoded from manual observations because we do not instrument it yet.
dexter_hypo_index_creations = {
    'tpch': 451,
    'job': 291,
    'tpcds': 3982
}
result = result.append({
    'Algorithm': styles['dexter'].label,
    'Runtime total': f"{s_to_m(25.17):.2f}",
    'Runtime index simulation': "-",
    'Runtime cost requests': "-",
    'Cost Requests Total': f"{2*len(QUERIES)}",
    'Not cached': 180,
    'Cost Requests Cached': f"{0.00:.2f}",
    'Evaluated Configurations': 2,
    'Indexes Simulated': f"{dexter_hypo_index_creations[BENCHMARK]}",
}, ignore_index=True)
result = result.sort_values(by=['Algorithm'])
result